In [1]:
from datasets import load_dataset
import nltk

[nltk_data] Downloading package punkt to /home/kk983/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
dataset = load_dataset('json', data_files='/share/dean/dataset/comments_awe.json')["train"]
bodies = dataset["body"]

Found cached dataset json (/home/kk983/.cache/huggingface/datasets/json/default-0df90851f4e7967e/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
bodies[0]
len(bodies)

197555

In [4]:
from nltk.tokenize import sent_tokenize, word_tokenize
sentences = [sent_tokenize(body) for body in bodies]
sentences = [sentence for doc in sentences for sentence in doc]

In [5]:
from sentence_transformers import SentenceTransformer

# Pre-calculate embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedding_model.encode(bodies, show_progress_bar=True)

Batches:   0%|          | 0/6174 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│   2                                                                                              │
│   3 # Pre-calculate embeddings                                                                   │
│   4 embedding_model = SentenceTransformer("all-MiniLM-L6-v2")                                    │
│ ❱ 5 embeddings = embedding_model.encode(bodies, show_progress_bar=True)                          │
│   6                                                                                              │
│                                                                                                  │
│ /home/kk983/.local/lib/python3.8/site-packages/sentence_transformers/SentenceTransformer.py:165  │
│ in encode                                                                                        │
│                                                                                                  │
│   162 │   │   │   features = batch_to_device(features, device)                                   │
│   163 │   │   │                                                                                  │
│   164 │   │   │   with torch.no_grad():                                                          │
│ ❱ 165 │   │   │   │   out_features = self.forward(features)                                      │
│   166 │   │   │   │                                                                              │
│   167 │   │   │   │   if output_value == 'token_embeddings':                                     │
│   168 │   │   │   │   │   embeddings = []                                                        │
│                                                                                                  │
│ /home/kk983/.local/lib/python3.8/site-packages/torch/nn/modules/container.py:217 in forward      │
│                                                                                                  │
│   214 │   # with Any as TorchScript expects a more precise type                                  │
│   215 │   def forward(self, input):                                                              │
│   216 │   │   for module in self:                                                                │
│ ❱ 217 │   │   │   input = module(input)                                                          │
│   218 │   │   return input                                                                       │
│   219 │                                                                                          │
│   220 │   def append(self, module: Module) -> 'Sequential':                                      │
│                                                                                                  │
│ /home/kk983/.local/lib/python3.8/site-packages/torch/nn/modules/module.py:1501 in _call_impl     │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                            

In [ ]:
import pickle

with open('embeddings.pickle', 'wb') as handle:
    pickle.dump(embeddings, handle)

In [ ]:
from umap import UMAP

umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=51)